In [40]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
from urllib import request as req
import math

In [2]:
soup_list = []
chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
dates = pd.date_range('08:00:00','09:59:00',freq='1min')
mins = range(len(dates))
for m in mins:
    hour = dates[0].hour + int(m / 60)
    minute = m % 60
    if hour < 10:
        hour = '0' + str(hour)
    if minute < 10:
        minute = '0' + str(minute)

    url = 'https://datalab.naver.com/keyword/realtimeList.naver'
    url += f'?datetime=2020-06-19T{hour}%3A{minute}%3A00&where=main'
    driver.get(url)
    soup_list.append(bs(driver.page_source,'html.parser'))
    time.sleep(0.1)
driver.close()

In [69]:
ranking_list = list(map(lambda i:i.select_one('.ranking_box'), soup_list))
col = [f'rank{i}' for i in range(1,21)]
df_ranking = pd.DataFrame(index=dates, columns=col)
df_rel_rank = pd.DataFrame(index=dates, columns=col)
for ranking_box, date in zip(ranking_list, dates):
    item_title = ranking_box.select('.item_title')
    item_title_list = list(map(lambda i:i.text, item_title))
    df_ranking.loc[date] = item_title_list
    
    item_sub_list_raw = list(map(lambda i:i.find_next_sibling(), item_title))
    item_sub_list = []
    for item_sub in item_sub_list_raw:
        if item_sub:
            item_sub_list.append(list(map(lambda i:i.replace('\n','').strip(), item_sub.text.split(','))))
        else:
            item_sub_list.append('')
    df_rel_rank.loc[date] = item_sub_list

In [70]:
df_ranking

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
2020-06-19 08:00:00,이강인,좌고우면,2020 6월 모의고사 등급컷,레알마드리드,주현정,안소현,메가스터디,ebsi,2020 고1 6월 모의고사,춘천 날씨,굿걸,경주 스쿨존 사고,로드 투 킹덤,토트넘 맨유,라이트 아웃,쌍갑포차,노수광,안소현 프로,외모지상주의,블루
2020-06-19 08:01:00,이강인,좌고우면,2020 6월 모의고사 등급컷,레알마드리드,안소현,주현정,메가스터디,2020 고1 6월 모의고사,굿걸,춘천 날씨,경주 스쿨존 사고,로드 투 킹덤,토트넘 맨유,라이트 아웃,학생 자가진단,쌍갑포차,노수광,안소현 프로,외모지상주의,블루
2020-06-19 08:02:00,이강인,좌고우면,2020 6월 모의고사 등급컷,레알마드리드,안소현,주현정,메가스터디,2020 고1 6월 모의고사,굿걸,춘천 날씨,경주 스쿨존 사고,로드 투 킹덤,토트넘 맨유,라이트 아웃,쌍갑포차,노수광,안소현 프로,블루,현대엘리베이터,외모지상주의
2020-06-19 08:03:00,이강인,좌고우면,2020 6월 모의고사 등급컷,레알마드리드,안소현,주현정,메가스터디,2020 고1 6월 모의고사,굿걸,춘천 날씨,로드 투 킹덤,경주 스쿨존 사고,영화 선물,토트넘 맨유,라이트 아웃,쌍갑포차,노수광,안소현 프로,블루,현대엘리베이터
2020-06-19 08:04:00,이강인,좌고우면,2020 6월 모의고사 등급컷,레알마드리드,안소현,영화 선물,ebsi,메가스터디,주현정,2020 고1 6월 모의고사,굿걸,춘천 날씨,로드 투 킹덤,경주 스쿨존 사고,토트넘 맨유,학생 자가진단,라이트 아웃,쌍갑포차,노수광,안소현 프로
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19 09:55:00,이강인,현대엘리베이터,2020 6월 모의고사 등급컷,우지윤,제니주노,좌고우면,퍼스텍,이강인 퇴장,모나미,미스 줄리,휴마시스,광진구청,에스에프씨,안소현,한일단조,ebsi,토트넘 맨유,한올바이오파마,아이크래프트,이다도시
2020-06-19 09:56:00,이강인,현대엘리베이터,2020 6월 모의고사 등급컷,우지윤,제니주노,좌고우면,퍼스텍,이강인 퇴장,시큐브,모나미,미스 줄리,휴마시스,광진구청,에스에프씨,안소현,한일단조,ebsi,sk증권우,토트넘 맨유,이다도시
2020-06-19 09:57:00,이강인,현대엘리베이터,2020 6월 모의고사 등급컷,우지윤,제니주노,좌고우면,이강인 퇴장,모나미,미스 줄리,휴마시스,광진구청,에스에프씨,안소현,sk증권우,ebsi,토트넘 맨유,이다도시,아이크래프트,장예은,현대바이오
2020-06-19 09:58:00,이강인,현대엘리베이터,2020 6월 모의고사 등급컷,우지윤,제니주노,좌고우면,이강인 퇴장,모나미,미스 줄리,휴마시스,광진구청,에스에프씨,안소현,한일단조,sk증권우,ebsi,토트넘 맨유,이다도시,아이크래프트,현대바이오


In [71]:
df_rel_rank

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
2020-06-19 08:00:00,[이강인 퇴장],,"[2020 6월 모의고사, 2020 6월 모의고사 답지, 등급컷]","[레알마드리드 발렌시아, 발렌시아]",,,,,,"[오늘 서울 날씨, 인천날씨, 파주 날씨, 서울 날씨, 서울오늘날씨, 평택 날씨, ...",,,,[토트넘],,,,,,
2020-06-19 08:01:00,[이강인 퇴장],,"[2020 6월 모의고사, 2020 6월 모의고사 답지, 등급컷]","[레알마드리드 발렌시아, 발렌시아]",,,[ebsi],,,"[오늘 서울 날씨, 인천날씨, 파주 날씨, 서울 날씨, 서울오늘날씨, 평택 날씨, ...",,,,,,,,,,
2020-06-19 08:02:00,[이강인 퇴장],,"[2020 6월 모의고사, 2020 6월 모의고사 답지, 등급컷]","[레알마드리드 발렌시아, 발렌시아]",,,[ebsi],,,"[오늘 서울 날씨, 인천날씨, 파주 날씨, 서울 날씨, 서울오늘날씨, 평택 날씨, ...",,,[토트넘],,,,,,,
2020-06-19 08:03:00,[이강인 퇴장],,"[2020 6월 모의고사, 2020 6월 모의고사 답지, 등급컷]","[레알마드리드 발렌시아, 발렌시아]",,,[ebsi],,,"[오늘 서울 날씨, 인천날씨, 파주 날씨, 서울 날씨, 서울오늘날씨, 수원 날씨, ...",,,[사라진 시간],[토트넘],,,,,,
2020-06-19 08:04:00,[이강인 퇴장],,"[2020 6월 모의고사, 2020 6월 모의고사 답지, 등급컷]","[레알마드리드 발렌시아, 발렌시아]",,[사라진 시간],,,,,,"[오늘 서울 날씨, 인천날씨, 파주 날씨, 서울 날씨, 서울오늘날씨, 수원 날씨, ...",,,[토트넘],,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19 09:55:00,,,"[2020 6월 모의고사 답지, 2020 6월 모의고사]",,,,,,,,,,,,,,,[신일제약],,
2020-06-19 09:56:00,,,"[2020 6월 모의고사 답지, 2020 6월 모의고사]",,,,,,,,,,,,,,,,,
2020-06-19 09:57:00,,"[퍼스텍, 한일단조, 휴니드, 스페코, 빅텍, 이디티]","[2020 6월 모의고사 답지, 2020 6월 모의고사]",,,,,,,,,,,,,,,,,
2020-06-19 09:58:00,,"[퍼스텍, 휴니드, 스페코, 빅텍]","[2020 6월 모의고사 답지, 2020 6월 모의고사]",,,,,,,,,,,,,,,,,


In [7]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)

In [8]:
jongmok_list = list(df[0]['회사명'])

In [101]:
df_ranking = df_ranking[df_ranking.isin(jongmok_list)]
set_ranking = set()
for date in dates:
    for ranking in df_ranking:
        data = df_ranking[ranking][date]
#         print(ranking, date)
        if type(data) is str:
            if data not in set_ranking:# and df_rel_rank[ranking][date] is not '':
                set_ranking.add(data)
#                 print(data)
                print(data, date)

메가스터디 2020-06-19 08:00:00
현대엘리베이터 2020-06-19 08:02:00
더존비즈온 2020-06-19 08:35:00
퍼스텍 2020-06-19 09:10:00
시큐브 2020-06-19 09:23:00
에스에프씨 2020-06-19 09:32:00
휴마시스 2020-06-19 09:34:00
한올바이오파마 2020-06-19 09:43:00
한일단조 2020-06-19 09:45:00
아이크래프트 2020-06-19 09:47:00
이디티 2020-06-19 09:52:00
모나미 2020-06-19 09:53:00
현대바이오 2020-06-19 09:57:00


In [98]:
df_ranking

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
2020-06-19 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,메가스터디,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-19 08:01:00,NaN,NaN,NaN,NaN,NaN,NaN,메가스터디,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-19 08:02:00,NaN,NaN,NaN,NaN,NaN,NaN,메가스터디,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현대엘리베이터,NaN
2020-06-19 08:03:00,NaN,NaN,NaN,NaN,NaN,NaN,메가스터디,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현대엘리베이터
2020-06-19 08:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메가스터디,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19 09:55:00,NaN,현대엘리베이터,NaN,NaN,NaN,NaN,퍼스텍,NaN,모나미,NaN,휴마시스,NaN,에스에프씨,NaN,한일단조,NaN,NaN,한올바이오파마,아이크래프트,NaN
2020-06-19 09:56:00,NaN,현대엘리베이터,NaN,NaN,NaN,NaN,퍼스텍,NaN,시큐브,모나미,NaN,휴마시스,NaN,에스에프씨,NaN,한일단조,NaN,NaN,NaN,NaN
2020-06-19 09:57:00,NaN,현대엘리베이터,NaN,NaN,NaN,NaN,NaN,모나미,NaN,휴마시스,NaN,에스에프씨,NaN,NaN,NaN,NaN,NaN,아이크래프트,NaN,현대바이오
2020-06-19 09:58:00,NaN,현대엘리베이터,NaN,NaN,NaN,NaN,NaN,모나미,NaN,휴마시스,NaN,에스에프씨,NaN,한일단조,NaN,NaN,NaN,NaN,아이크래프트,현대바이오
